In [1]:
import sys,os
sys.path.insert(0, os.path.abspath(".."))

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import os

In [15]:
import pycaret
import sklearn
import pandas as pd
print(f'pandas:{pd.__version__}')
print(f'sklearn:{sklearn.__version__}')
print(f'pycaret:{pycaret.__version__}')

pandas:2.1.4
sklearn:1.4.2
pycaret:3.3.2


In [4]:
target='homo_lumo_gap_min'
cfid_path = f'../results_2/2dmd/CatBoostRegressor/{target}/cfid/all_density/'
X_train = pd.read_csv(os.path.join(cfid_path, 'X_train.csv'),index_col=0)
X_test = pd.read_csv(os.path.join(cfid_path, 'X_test.csv'),index_col=0)
y_train = pd.read_csv(os.path.join(cfid_path, 'y_train.csv'),index_col=0)
y_test = pd.read_csv(os.path.join(cfid_path, 'y_test.csv'),index_col=0)

In [5]:

sample_weight = pd.read_csv(os.path.join(cfid_path, 'sample_weight.csv'),index_col=0)

In [6]:
test_data = pd.merge(X_test, y_test, left_index=True, right_index=True)
train_data = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [7]:
from pycaret.regression import *
s = RegressionExperiment()
s.setup(data=train_data, target = target,
         session_id = 42,
        train_size=0.8,fold=5, #feature_selection=True,
        n_jobs=-1)

,Description,Value
0,Session id,42
1,Target,homo_lumo_gap_min
2,Target type,Regression
3,Original data shape,"(11892, 1558)"
4,Transformed data shape,"(11892, 1558)"
5,Transformed train set shape,"(9513, 1558)"
6,Transformed test set shape,"(2379, 1558)"
7,Numeric features,1557
8,Preprocess,True
9,Imputation type,simple


In [8]:
best =s.compare_models(n_select=10, sort = "MAE", include=[
        'et', 'rf', 'catboost', 'lightgbm',
    'dt', 'gbr', 'knn', 'ada', 'lr'
])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0484,0.0106,0.1029,0.9664,0.0682,0.1846,14.1240
rf,Random Forest Regressor,0.0493,0.0107,0.1034,0.9660,0.0681,0.1852,20.6800
lightgbm,Light Gradient Boosting Machine,0.0535,0.0112,0.1056,0.9646,0.0697,0.1976,2.5920
catboost,CatBoost Regressor,0.0540,0.0108,0.1038,0.9659,0.0688,0.2035,28.3400
dt,Decision Tree Regressor,0.0591,0.0177,0.1329,0.9439,0.0874,0.2124,0.7900
gbr,Gradient Boosting Regressor,0.0654,0.0132,0.1150,0.9581,0.0757,0.2461,8.7100
knn,K Neighbors Regressor,0.1010,0.0388,0.1967,0.8771,0.1141,0.3438,0.9140
ada,AdaBoost Regressor,0.1284,0.0292,0.1702,0.9072,0.1120,0.5427,4.0640
lr,Linear Regression,0.3089,0.1724,0.4124,0.4555,0.2223,0.8829,0.8960


In [9]:
from sklearn.metrics import mean_absolute_error
trained_models = []
weighted_mae_results = []
for model in best:
    finalize = s.finalize_model(model)
    trained_models.append(finalize)



In [10]:
weighted_mae_results=[]
for trained_model in trained_models:
    y_pred_test = trained_model.predict(X_test)
    y_pred_train = trained_model.predict(X_train)
    weighted_mae_test = mean_absolute_error(y_test,y_pred_test,sample_weight=sample_weight['weight'].loc[y_test.index])
    weighted_mae_train = mean_absolute_error(y_train,y_pred_train,sample_weight=sample_weight['weight'].loc[y_train.index])
    test_train_ratio = weighted_mae_test/weighted_mae_train
    weighted_mae_results.append({
        'Model': type(trained_model.steps[-1][-1]).__name__,
        'test-Weighted_MAE': weighted_mae_test,
        'train-Weighted_MAE': weighted_mae_train,
        'test_train_ratio': test_train_ratio
    })
    

In [11]:
results_df = pd.DataFrame(weighted_mae_results).sort_values(by='test-Weighted_MAE')
results_df.round(6)

,Model,test-Weighted_MAE,train-Weighted_MAE,test_train_ratio
0,ExtraTreesRegressor,0.103717,0.001777,58.368514
1,RandomForestRegressor,0.107324,0.043893,2.445132
2,LGBMRegressor,0.108032,0.071759,1.505499
3,CatBoostRegressor,0.110342,0.065584,1.682466
5,GradientBoostingRegressor,0.122135,0.115519,1.057270
4,DecisionTreeRegressor,0.141212,0.001777,79.469300
6,KNeighborsRegressor,0.171929,0.146215,1.175859
7,AdaBoostRegressor,0.212580,0.209102,1.016634
8,LinearRegression,0.331547,0.344950,0.961146
